In [2]:
#markov similarity

In [ ]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkConf
sc = SparkContext("spark://39.99.129.214:7077", "SecondApp")
spark = SparkSession.builder.master("spark://39.99.129.214:7077").config(conf=SparkConf()).getOrCreate()

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pymongo
import pandas as pd
from pyspark.sql import SQLContext
from pyspark import SparkContext
from tqdm import tqdm
import copy

'''
specify collection name, here:
    ratings_temp: original ml-20m ratings
    ratings_sort: sort ratings_temp by userId (ascend) and timestamp (ascend)
'''
def read_mongo(collection):
    client = pymongo.MongoClient("39.98.136.173", 9099)
    client.movie.authenticate('user','cloud',mechanism='SCRAM-SHA-1')
    database = client['movie']
    ratings = database['ratings_sort']
    cursor = ratings.find()
    cnt = 0
    #df = pd.DataFrame(columns=('userId','movieId','rating','timestamp'))
    df = [] 
    for i in tqdm(range(2000000)):
        #userId = copy.deepcopy(cursor[i]['userId'])
        movieId = cursor[i]['movieId']
        #rating = copy.deepcopy(cursor[i]['rating'])
        #timestamp = copy.deepcopy(cursor[i]['timestamp'])
    #         print(userId,movieId,rating,timestamp)
    #         new = pd.DataFrame({'userId':[userId],'movieId':[movieId],'rating':[rating],'timestamp':[timestamp]})
    #         print(new)
        #df.append(pd.DataFrame(record,index=[0]),ignore_index=True)
        df.append([movieId])
        cnt = cnt + 1
        if cnt == 2000000:
            break
    
#     df = pd.DataFrame(tqdm(list(cursor)))
#     df = df[['userId', 'movieId', 'rating', 'timestamp']]
#     spark = SparkSession.builder.appName('readMongo').getOrCreate()
#     sqlContest = SQLContext(spark)
#     print("creating spark dataframe......")
#     spark_df = sqlContest.createDataFrame(df)
#     return spark_df
df = read_mongo('ratings_sort')

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('/usr/dataset/ml-20m/ratings.csv')
df['userId'] = df['movieId'].shift(1)
df = df.fillna(1)
df['userId'] = np.array(df['userId'].values,dtype=np.int)
df['rating'] = np.random.randint(0,2,len(df))
df[['userId','movieId','rating']].to_csv('/usr/dataset/ml-20m/markov.csv')

In [ ]:
len(df)

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('RS').getOrCreate()
# load dataset
df = spark.read.csv('/usr/dataset/ml-20m/markov.csv', inferSchema=True, header=True)



In [2]:
df.show(10)

+---+------+-------+------+
|_c0|userId|movieId|rating|
+---+------+-------+------+
|  0|     1|      2|     0|
|  1|     2|     29|     1|
|  2|    29|     32|     1|
|  3|    32|     47|     0|
|  4|    47|     50|     0|
|  5|    50|    112|     0|
|  6|   112|    151|     1|
|  7|   151|    223|     1|
|  8|   223|    253|     1|
|  9|   253|    260|     1|
+---+------+-------+------+
only showing top 10 rows



In [3]:
userCount_df = df.groupBy('userId').count().orderBy('count', ascending=False)
# Spark's DataFrame --> Pandas's DataFrame
user_df = userCount_df.toPandas()

In [4]:
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, IndexToString
stringIndexer = StringIndexer(inputCol='movieId', outputCol='movieId_num')
model = stringIndexer.fit(df)
new_df  = model.transform(df)
new_df.show(10)

+---+------+-------+------+-----------+
|_c0|userId|movieId|rating|movieId_num|
+---+------+-------+------+-----------+
|  0|     1|      2|     0|      125.0|
|  1|     2|     29|     1|      564.0|
|  2|    29|     32|     1|       19.0|
|  3|    32|     47|     0|       23.0|
|  4|    47|     50|     0|       14.0|
|  5|    50|    112|     0|      370.0|
|  6|   112|    151|     1|      335.0|
|  7|   151|    223|     1|      106.0|
|  8|   223|    253|     1|       77.0|
|  9|   253|    260|     1|        5.0|
+---+------+-------+------+-----------+
only showing top 10 rows



In [5]:
train_df, test_df = new_df.randomSplit([0.7, 0.3])

print('train_df, (%d, %d)'%(train_df.count(), len(train_df.columns)))
print('test_df, (%d, %d)'%(test_df.count(), len(test_df.columns)))


train_df, (13996257, 5)
test_df, (6004006, 5)


In [6]:
from pyspark.ml.recommendation import ALS
#train for model
rec = ALS(maxIter=10, regParam=0.01, userCol='userId', itemCol='movieId_num', ratingCol='rating', nonnegative=True,
                 coldStartStrategy='drop')
rs_model = rec.fit(train_df)


In [7]:
#test for model
test = test_df.select(['userId','movieId','movieId_num'])
test_pred = rs_model.transform(test)
test_pred.show()


+------+-------+-----------+----------+
|userId|movieId|movieId_num|prediction|
+------+-------+-----------+----------+
|  1591|     17|      148.0|  0.466884|
|  1127|     17|      148.0|0.45668304|
|  2563|     17|      148.0|0.46466967|
|  4161|     17|      148.0|0.47137365|
|  4929|     17|      148.0|0.51497185|
| 48780|     17|      148.0|0.45662728|
| 56941|     17|      148.0| 0.5391659|
|  2996|     17|      148.0|0.46675533|
|  6482|     17|      148.0|0.49000967|
| 76293|     17|      148.0| 0.5333507|
|  8665|     17|      148.0|0.46893552|
|101142|     17|      148.0|0.44186097|
|128830|     17|      148.0|0.26811567|
|  5995|     17|      148.0| 0.4856425|
|  6722|     17|      148.0| 0.5723945|
|  6832|     17|      148.0|0.46817586|
| 27822|     17|      148.0|0.46095657|
|  1466|     17|      148.0| 0.4438986|
|  4396|     17|      148.0|0.44765642|
|120132|     17|      148.0|0.19979942|
+------+-------+-----------+----------+
only showing top 20 rows



In [8]:
#test rmse for model
from pyspark.ml.evaluation import RegressionEvaluator

test_pred = rs_model.transform(test_df)

evaluate_result = RegressionEvaluator(metricName='rmse', predictionCol='prediction', labelCol='rating')
rmse = evaluate_result.evaluate(test_pred)
print('test rmse is %f'%rmse)


test rmse is 0.502530


In [9]:
#e.g. recommend movie for user 66
# total movies
nunique_movies = new_df.select('movieId').distinct()
a = nunique_movies.alias('a')
user_id = 66
# user_id = 66，看过的电影
watched_movies = new_df.filter(new_df['userId'] == user_id).select('movieId').distinct()
b = watched_movies.alias('b')
# a join b
total_movies = a.join(b, a.movieId == b.movieId, how='left')
user_66_not_watched_movies = total_movies.where(col('b.movieId').isNull()).select(a.movieId).distinct()
user_66_not_watched_movies = user_66_not_watched_movies.withColumn('userId', lit(int(user_id)))
user_66_not_watched_movies.show(10, False)

+-------+------+
|movieId|userId|
+-------+------+
|148    |66    |
|463    |66    |
|471    |66    |
|496    |66    |
|833    |66    |
|1088   |66    |
|1238   |66    |
|1342   |66    |
|1580   |66    |
|1591   |66    |
+-------+------+
only showing top 10 rows



In [10]:
#recommendation for all users
rec = rs_model.recommendForAllUsers(10)

In [18]:
df_rec= rec.toPandas()

In [19]:
df_rec = df_rec.sort_values(by='userId')

In [20]:
df_rec

,userId,recommendations
5355,1,"[(14672, 1.1313905715942383), (17758, 1.082917..."
22045,2,"[(18198, 1.1249234676361084), (18792, 1.076898..."
6363,3,"[(18804, 1.0842939615249634), (18198, 1.082339..."
12871,4,"[(18198, 1.1037191152572632), (18804, 1.073117..."
8245,5,"[(21832, 1.0734281539916992), (18804, 1.064790..."
...,...,...
11862,131248,"[(13951, 1.4968010187149048), (15278, 1.471555..."
3219,131256,"[(0, 0.0), (10, 0.0), (20, 0.0), (30, 0.0), (4..."
24171,131258,"[(0, 0.0), (10, 0.0), (20, 0.0), (30, 0.0), (4..."
15295,131260,"[(18792, 2.265856981277466), (14672, 2.2063508..."


In [29]:
import numpy as np
i = 0
res = []
for lines in df_rec.values:
    userID = lines[0]
    items = [line[0] for line in lines[1]]
    while userID != i:
        res.append(np.random.randint(0,131262,10).tolist())
        i = i + 1
    
    res.append(items)
    i = i +1 


In [30]:
import pickle
fw = open('markov_similarity.pkl','wb')  
pickle.dump(res, fw, -1)  


131263